In [ ]:
''' using MNIST dataset.

 99.25% test accuracy
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save the model
model.save('mnistCNN.h5')



In [1]:
#FOR IMAGE SAMPLES

import cv2
import numpy as np
from keras.models import load_model

model = load_model('mnistCNN.h5')
#model = load_model('digit_classifier2.h5')


def get_numbers(y_pred):
    for number, per in enumerate(y_pred[0]):
        if per != 0:
            final_number = str(int(number))
            per = round((per * 100), 2)
            return final_number, per

        
#img=cv2.imread("MNISTsamples.png")
img=cv2.imread("indir14.png")


img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
img_gau = cv2.GaussianBlur(img_gray, (5, 5), 0)
ret, thresh = cv2.threshold(img_gau, 80, 255, cv2.THRESH_BINARY_INV)
#cv2.imshow("Frame thersh",thresh)

kernel = np.ones((5, 5), np.uint8)
dilation = cv2.dilate(thresh, kernel, iterations=1)
#cv2.imshow("Frame dilation", thresh)

edged = cv2.Canny(dilation, 50, 250)
#cv2.imshow("Frame edged", thresh)

#key = cv2.waitKey(0)
'''if key == 27:
    break
elif key & 0xFF == ord('c'):'''

cv2.imwrite('capture.jpg',img)
capture_img = cv2.imread('capture.jpg')

img2 = capture_img.copy()
img_gray = cv2.cvtColor(capture_img, cv2.COLOR_RGB2GRAY)
img_gau = cv2.GaussianBlur(img_gray, (5, 5), 0)
ret, thresh = cv2.threshold(img_gau, 80, 255, cv2.THRESH_BINARY_INV)

kernel = np.ones((5, 5), np.uint8)
dilation = cv2.dilate(thresh, kernel, iterations=1)

edged = cv2.Canny(dilation, 50, 250)

contours, hierachy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

num_str = ''
per = ''

num_list = []

if len(contours) > 0:
    for c in contours:
        if cv2.contourArea(c) > 2500:
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(img2, (x, y), (x + w, y + h), (255, 0, 255), 3)

            new_img = thresh[y:y + h, x:x + w]
            new_img2 = cv2.resize(new_img, (28, 28))
            im2arr = np.array(new_img2)
            im2arr = im2arr.reshape(1, 28, 28, 1)
            y_pred = model.predict(im2arr)

            num, per = get_numbers(y_pred)
            num_list.append(str(int(num)))
            num_str = '[' + str(str(int(num))) + ']'
            cv2.putText(img2, num_str, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)

str1 = ' '.join(num_list)
if (str1 != ''):
    y_p = str('Predicted Value is ' + str(str1))
    print(y_p)
    cv2.putText(img2, y_p, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
cv2.imshow("Capture Frame", img2)
#cv2.imshow("Contours Frame", thresh)
cv2.waitKey(0)

Using TensorFlow backend.
W0912 10:37:06.965173  2440 deprecation_wrapper.py:119] From C:\Users\fatma\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0912 10:37:06.988058  2440 deprecation_wrapper.py:119] From C:\Users\fatma\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0912 10:37:07.024074  2440 deprecation_wrapper.py:119] From C:\Users\fatma\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0912 10:37:07.024074  2440 deprecation_wrapper.py:119] From C:\Users\fatma\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0912 10:37:07.028051  2440 deprecation_wrapper.py:1

Predicted Value is 5 3 5 4 3 3


-1

In [ ]:
#FOR VIDEO SAMPLES
import cv2
import numpy as np
from keras.models import load_model

model = load_model('mnistCNN.h5')

def get_numbers(y_pred):
    for number, per in enumerate(y_pred[0]):
        if per != 0:
            final_number = str(int(number))
            per = round((per * 100), 2)
            return final_number, per

#cameradan girdi için "project.mp4" yazan kısma 0 yazın
video = cv2.VideoCapture("project.mp4")
if(video.isOpened()):
    while True:
        check, img = video.read()
        cv2.imshow("Frame", img)

        #Display purpose
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img_gau = cv2.GaussianBlur(img_gray, (5, 5), 0)
        ret, thresh = cv2.threshold(img_gau, 80, 255, cv2.THRESH_BINARY_INV)
        cv2.imshow("Frame thersh",thresh)

        kernel = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(thresh, kernel, iterations=1)
        cv2.imshow("Frame dilation", thresh)

        edged = cv2.Canny(dilation, 50, 250)
        cv2.imshow("Frame edged", thresh)

        key = cv2.waitKey(1)

        if key == 27:
            break
        elif key & 0xFF == ord('c'):
            cv2.imwrite('capture.jpg',img)
            capture_img = cv2.imread('capture.jpg')

            img2 = capture_img.copy()
            img_gray = cv2.cvtColor(capture_img, cv2.COLOR_RGB2GRAY)
            img_gau = cv2.GaussianBlur(img_gray, (5, 5), 0)
            ret, thresh = cv2.threshold(img_gau, 80, 255, cv2.THRESH_BINARY_INV)

            kernel = np.ones((5, 5), np.uint8)
            dilation = cv2.dilate(thresh, kernel, iterations=1)

            edged = cv2.Canny(dilation, 50, 250)

            contours, hierachy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            num_str = ''
            per = ''

            num_list = []
            if len(contours) > 0:
                for c in contours:
                    if cv2.contourArea(c) > 2500:
                        x, y, w, h = cv2.boundingRect(c)
                        cv2.rectangle(img2, (x, y), (x + w, y + h), (255, 0, 255), 3)

                        new_img = thresh[y:y + h, x:x + w]
                        new_img2 = cv2.resize(new_img, (28, 28))
                        im2arr = np.array(new_img2)
                        im2arr = im2arr.reshape(1, 28, 28, 1)
                        y_pred = model.predict(im2arr)

                        num, per = get_numbers(y_pred)
                        num_list.append(str(int(num)))
                        num_str = '[' + str(str(int(num))) + ']'
                        cv2.putText(img2, num_str, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)

            str1 = ' '.join(num_list)
            if (str1 != ''):
                y_p = str('Predicted Value is ' + str(str1))
                print(y_p)
                cv2.putText(img2, y_p, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.imshow("Capture Frame", img2)
            cv2.imshow("Contours Frame", thresh)

video.release()
cv2.destroyAllWindows()